互联网中充斥着大量钓鱼欺诈类网站。这类非法网站通常试图掩人耳目、充当正规网站，而实际上却是在窃取用户的身份、密码、交易等重要信息
机器学习在信息安全领域中的一个重要应用就是用来识别这些钓鱼网站。fraudulent.csv在data文件夹中。

fraudulent.csv文件中含有10,086条数据，每条数据含有18个特征以及1个标签。各个特征的含义如下：

    contain_IP：网址中是否包涵ip，比如http://121.99.3.123/fake.html 包含ip。1表示包含，0表示不包含；
    is_long：网址字符是否过长。1表示网址过长，0表示网址不长；
    is_tinyurl：网址是否是短网址。比如https://bit.ly/2kXX6jV 就是短网址。1表示是短网址，0表示不是；
    contain_at：网址是否包含“@”符号。1表示包含，0表示不包含；
    contain_double_slash：网址是否包含“//”符号，该符号用来表示网址跳转。1表示包含，0表示不包含；
    contain_dash：网址是否包含“-”符号，该符号经常帮助用来伪装真网站，比如www.my-taobao.com 。 1表示包含，0表示不包含；
    contain_subdomain：网址是否包含子域名，比如www.ecnu.edu.cn 就包含edu和cn子域名。1表示包含，0表示不包含；
    is_SSL：网址是否是https安全链接。1表示包含，0表示不包含；
    with_long_history：网址所属的主域名存在的时间。1表示长久，0表示不长久；
    contain_icon：网址网页是否有小图标。1表示包含，0表示不包含；
    contain_ext_domain：该网页是否加载其他域名下的附件或者网页。1表示包含，0表示不包含；
    contain_email_to：该网页是否包含发送邮件的组件。1表示包含，0表示不包含；
    allow_right_click：该网页是否允许用户进行右击操作。1表示允许，0表示不允许；
    contain_pop_up_windowL：该网页是否包含弹窗。1表示包含，0表示不包含；
    contain_Iframe：该网页是否包含Iframe（嵌套网页）。1表示包含，0表示不包含；
    has_DNSRecord：网址是否有DNS记录。1表示有，0表示无；
    traffic：该网站的流量大小。1表示大，0表示小；
    google_rank：该网址在google搜索中的排名。1表示高于同类网站的平均排名，0表示低于同类网站的平均排名；
    
    y：表示网站是否是钓鱼欺诈网站，1表示是，0表示不是。
    原始数据中含有大量缺失值，请自行处理这些缺失值（可以剔除缺失值过多的列或者使用众数填充等方法）。

将原始数据分为训练集、测试集（随机种子请设置为1）（若有需要可以将训练集进一步分为训练集和验证集）。

现在请建立一个二分类模型，使用训练集训练模型，再使用测试集测试模型。

评估指标为F1值

分类模型可采用：k-近邻、决策树、逻辑回归、支持向量机等。

可以与周围同学比较一下F1值的大小（越接近1越好），看看谁的数据预处理和分类模型更强。

In [1]:
import pandas as pd
data_path = "data/fraudulent.csv"
df = pd.read_csv(data_path)

#print(df.head())  # 查看前5行数据
#print(df.info())  # 查看列的类型和缺失值情况
#print(df.describe())  # 查看数值特征的统计信息

#print(df['y'].value_counts())  # 查看标签分布

# print(df.isnull().sum())  # 每列的缺失值数量

# 剔除缺失值多的列
missing_threshold = 0.5  # 设置阈值
df = df[df.columns[df.isnull().mean() < missing_threshold]]

# 用众数填充
df = df.fillna(df.mode().iloc[0])

# print(df.isnull().sum())  # 确认无缺失值

from sklearn.model_selection import train_test_split

X = df.drop(columns=['y'])  # 特征
y = df['y']  # 标签

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

model = LogisticRegression(random_state=1, max_iter=500)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
from sklearn.metrics import f1_score

f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")
models = {
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(random_state=1),
    'Logistic Regression': LogisticRegression(random_state=1, max_iter=500),
    'SVM': SVC(random_state=1)
}

for name, clf in models.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    print(f"{name}: F1 Score = {f1:.4f}")

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier  # 示例模型

# 定义参数网格
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 初始化模型
rf = RandomForestClassifier(random_state=1)

# 网格搜索
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='f1', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

# 最佳参数和得分
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)



F1 Score: 0.8491
KNN: F1 Score = 0.8385
Decision Tree: F1 Score = 0.8667
Logistic Regression: F1 Score = 0.8491
SVM: F1 Score = 0.8585
Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Best F1 Score: 0.8668632712843596
